In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
%run PARAMETERS.py
NAME = 'APG'
MCMC_STEPS = 5
PATH = NAME + '-bmnist-%dsamples' % (SAMPLE_SIZE)
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

In [4]:
from model_operations import Init_models, Save_models
from crop import *

models, optimizer = Init_models(K=K,
                                D=D,
                                FRAME_PIXELS=FRAME_PIXELS, 
                                DIGIT_PIXELS=DIGIT_PIXELS, 
                                HIDDEN_LIST=HIDDEN_LIST, 
                                Sigma0=Sigma0,
                                CUDA=CUDA, 
                                device=DEVICE, 
                                lr=LEARNING_RATE, 
                                RESTORE=False, 
                                PATH=PATH)
crop = Crop(digit_size=28, 
            frame_size=64, 
            CUDA=CUDA, 
            device=DEVICE)

In [ ]:
from apg import APG
train(models=models, 
      objective=APG, 
      optimizer=optimizer, 
      data_path=data_path, 
      mcmc_steps=MCMC_STEPS,
      mnist_mean=mnist_mean,
      crop=crop,
      Train_Params=Train_Params,
      CUDA=CUDA,
      device=DEVICE,
      path=PATH)

Completed  in (132s),  phi_loss: -62.281,  theta_loss: 19000.575,  ess: 1.045
Completed  in (134s),  phi_loss: -65.343,  theta_loss: 18958.704,  ess: 1.046
Completed  in (132s),  phi_loss: -69.073,  theta_loss: 18961.124,  ess: 1.050
Completed  in (132s),  phi_loss: -66.556,  theta_loss: 19009.732,  ess: 1.047
Completed  in (133s),  phi_loss: -69.117,  theta_loss: 18773.351,  ess: 1.049
Completed  in (133s),  phi_loss: -69.822,  theta_loss: 18701.833,  ess: 1.046
Completed  in (132s),  phi_loss: -68.981,  theta_loss: 18770.548,  ess: 1.046
Completed  in (132s),  phi_loss: -70.363,  theta_loss: 18825.449,  ess: 1.046
Completed  in (133s),  phi_loss: -70.890,  theta_loss: 18891.125,  ess: 1.049
Completed  in (133s),  phi_loss: -69.961,  theta_loss: 18832.224,  ess: 1.047


In [ ]:
Save_models(models, PATH=PATH)